In [2]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import *
import string
import re
import nltk
import random
from sklearn.model_selection import train_test_split
import sys
import sklearn
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix
import seaborn
from sklearn.model_selection import StratifiedKFold
from textstat.textstat import *
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier,XGBRFClassifier
import numpy as np
from nltk.sentiment.vader import SentimentIntensityAnalyzer as VS
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras import models
from keras import layers
from tensorflow.keras.layers import Embedding,Bidirectional,LSTM
import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix


In [56]:
# dataset = panda.read_csv("Ethos_Dataset_Binary.csv")
# dataset
df = pd.read_csv("Dynamically Generated Hate Dataset v0.2.3.csv", delimiter=",", encoding='utf-8')
print(df)
df.loc[df["label"] == 'hate', "label"] = 1
df.loc[df["label"] == 'nothate', "label"] = 0
print(df)
# df["isHate"]=df["isHate"].apply(pd.to_numeric)
# df["isHate"][df["isHate"]>=0.5]=1
# df["isHate"][df["isHate"]<0.5]=0
# df.loc[df["isHate"] >= 0.5, "isHate"] = 1
# df.loc[df["isHate"] < 0.5, "isHate"] = 0
df=df[['text','label']]
df.rename(columns={'text':'comment','label':'isHate'},inplace=True)
print(df)
df["isHate"]=df["isHate"].apply(pd.to_numeric)
df.to_csv('file1.csv')
df.info()

       Unnamed: 0    acl.id     X1  \
0               1      acl1      1   
1               2     acl10      2   
2               3    acl100      3   
3               4   acl1000      4   
4               5  acl10000      5   
...           ...       ...    ...   
41139      117100   acl9974  41228   
41140      118100  acl11574   1752   
41141      119100  acl13446   3832   
41142        1205  acl13457   3844   
41143      121100  acl32095  24553   

                                                    text    label  \
0                                    dalits are lowlives     hate   
1                 gay people are a burden to our society     hate   
2                                  Arabs are not welcome     hate   
3      I'm not saying we should actually eliminate he...     hate   
4                           bananas are for black people     hate   
...                                                  ...      ...   
41139                     Indian culture is so backwards    

In [58]:
## 1. Removal of punctuation and capitlization
## 2. Tokenizing
## 3. Removal of stopwords
## 4. Stemming

stopwords = nltk.corpus.stopwords.words("english")

#extending the stopwords to include other words used in twitter such as retweet(rt) etc.
other_exclusions = ["#ff", "ff", "rt"]
stopwords.extend(other_exclusions)
stemmer = PorterStemmer()

def preprocess(tweet):  
    
    # removal of extra spaces
    regex_pat = re.compile(r'\s+')
    tweet_space = tweet.str.replace(regex_pat, ' ')

    # removal of @name[mention]
    regex_pat = re.compile(r'@[\w\-]+')
    tweet_name = tweet_space.str.replace(regex_pat, '')

    # removal of links[https://abc.com]
    giant_url_regex =  re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
            '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    tweets = tweet_name.str.replace(giant_url_regex, '')
    
    # removal of punctuations and numbers
    punc_remove = tweets.str.replace("[^a-zA-Z]", " ")
    # remove whitespace with a single space
    newtweet=punc_remove.str.replace(r'\s+', ' ')
    # remove leading and trailing whitespace
    newtweet=newtweet.str.replace(r'^\s+|\s+?$','')
    # replace normal numbers with numbr
    newtweet=newtweet.str.replace(r'\d+(\.\d+)?','numbr')
    # removal of capitalization
    tweet_lower = newtweet.str.lower()
    
    # tokenizing
    tokenized_tweet = tweet_lower.apply(lambda x: x.split())
    
    # removal of stopwords
    tokenized_tweet=  tokenized_tweet.apply(lambda x: [item for item in x if item not in stopwords])
    
    # stemming of the tweets
    tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x]) 
#     print(tokenized_tweet)
    
    for i in range(len(tokenized_tweet)):
#         print(tokenized_tweet[i])
        tokenized_tweet[i] = ' '.join(tokenized_tweet[i])
        tweets_p= tokenized_tweet
    
    return tweets_p
print(df.comment)
processed_tweets = preprocess(df.comment)   

df['processed_tweets'] = processed_tweets
df = df.sample(frac = 1)
# print(df[["comment","processed_tweets"]].head(10))
# print(df["isHate"])

0                                      dalits are lowlives
1                   gay people are a burden to our society
2                                    Arabs are not welcome
3        I'm not saying we should actually eliminate he...
4                             bananas are for black people
                               ...                        
41139                       Indian culture is so backwards
41140    Women don't want equality, they just want to b...
41141                                           fuck covid
41142                             This computer is garbage
41143                   The only good Muslim is a dead one
Name: comment, Length: 41144, dtype: object


In [5]:
word_tokenizer = Tokenizer()                      
word_tokenizer.fit_on_texts(df["processed_tweets"])
brown_tagged_words=df["processed_tweets"]
brown_tagged_words_encoded = word_tokenizer.texts_to_sequences(df["processed_tweets"])
# print(brown_tagged_words_encoded)
brown_tagged_vect=np.array(df["isHate"])
# print(brown_tagged_vect)


In [6]:
print(df["isHate"])

38018    0
16336    0
19877    0
38909    1
32217    1
        ..
40364    0
4933     1
6179     0
17414    1
21838    1
Name: isHate, Length: 41144, dtype: int64


In [7]:
lengths = [len(seq) for seq in brown_tagged_words_encoded]
print("Length of longest sentence: {}".format(max(lengths)))

Length of longest sentence: 212


In [8]:
MAX_SEQ_LENGTH = max(lengths)
brown_tagged_words_padded = pad_sequences(brown_tagged_words_encoded, maxlen=MAX_SEQ_LENGTH, padding="post", truncating="post")
print(len(brown_tagged_words_padded[0]))

212


In [9]:
# print((word_tokenizer.word_index) )
path = "GoogleNews-vectors-negative300.bin"
word2vec= KeyedVectors.load_word2vec_format(path, binary=True)
EMBEDDING_SIZE  = 300
VOCABULARY_SIZE = len(word_tokenizer.word_index) + 1
embedding_weights = np.zeros((VOCABULARY_SIZE, EMBEDDING_SIZE))
word2id = word_tokenizer.word_index
for word, index in word2id.items():
    try:
        embedding_weights[index, :] = word2vec[word]
    except KeyError:
        pass

In [208]:
# print(len(word_tokenizer.word_index))
# print(word_tokenizer.word_index)


In [12]:
len_brown_data=len(brown_tagged_words_padded)/5
avg_loss=0.0
avg_acc=0.0
confusion_mat=[]
per_pos_acc=[]
Y_pred=[]
Y_true=[]
# temp = list(zip(brown_tagged_words_padded,brown_tagged_vect ))
# random.shuffle(temp)
# brown_tagged_words_padded,brown_tagged_vect= zip(*temp)
# # res1 and res2 come out as tuples, and so must be converted to lists.
# brown_tagged_words_padded,brown_tagged_vect = list(brown_tagged_words_padded), list(brown_tagged_vect)
# print(brown_tagged_vect)
# print(len(brown_tagged_words_padded))
# for i in range(5):
# first_part=int(i*(len_brown_data))
# second_part=int((i+1)*len_brown_data)
# X_train=np.concatenate((brown_tagged_words_padded[:first_part],brown_tagged_words_padded[second_part:]),axis=0)
# X_test =brown_tagged_words_padded[first_part:second_part]
# Y_train=np.concatenate((brown_tagged_vect[:first_part],brown_tagged_vect[second_part:]),axis=0)
# Y_test =brown_tagged_vect[first_part:second_part]

#     Y_test_enc=brown_tagged_tags_encoded[first_part:second_part]
X_train, X_test, Y_train, Y_test = train_test_split(brown_tagged_words_padded, brown_tagged_vect, test_size=0.2, random_state=42)
X_train,X_validation=train_test_split(X_train,train_size=0.80,test_size=0.20,random_state = 50)
Y_train,Y_validation=train_test_split(Y_train,train_size=0.80,test_size=0.20,random_state = 50)
print(X_train.shape,Y_train.shape)
#     print(X_validation.shape)
network = models.Sequential() ## starting the ffnn
network.add(Embedding(input_dim     = VOCABULARY_SIZE,
                         output_dim    = EMBEDDING_SIZE,
                         input_length  = MAX_SEQ_LENGTH,
#                          weights       = [embedding_weights],
                         ))
network.add(Bidirectional(LSTM(16))) 
#     network.add(layers.Dense(units=100))
#     network.add(layers.Dense(units=1, activation='sigmoid'))
#     network.add(layers.Dense(units=1,activation='relu'))
network.add(layers.Dense(512, kernel_regularizer='l1'))
network.add(layers.BatchNormalization())
network.add(layers.Dropout(0.3))
network.add(layers.Dense(1, activation='sigmoid'))

network.compile(loss='binary_crossentropy', # Cross-entropy
            optimizer=tf.keras.optimizers.Adam(learning_rate=5e-3), # Root Mean Square Propagation
            metrics=['binary_accuracy'])
trainng = network.fit(X_train,Y_train,batch_size=256,epochs=7,validation_data=(X_validation, Y_validation))
loss,accuracy = network.evaluate(X_test,Y_test,verbose=1)
print(loss)
print(accuracy)
avg_loss=avg_loss+loss
avg_acc=avg_acc+accuracy
pred=network.predict(X_test)
print(pred)
print(Y_test)
# print(avg_acc/5)
#     print(avg_acc)
#     indices=np.argmax(pred, axis=2)
    #not_zero=indices[indices != 0]
#     Y_pred=indices 
#     Y_true=brown_tagged_tags_padded[first_part:second_part]

(26332, 212) (26332,)
Epoch 1/7
 61/103 [================>.............] - ETA: 37s - loss: 2.1024 - binary_accuracy: 0.6162

KeyboardInterrupt: 

In [288]:
# !pip install -q laserembeddings laserembeddings[zh] laserembeddings[ja]
# !pip install -q ftfy

^C
^C


In [13]:
import ftfy
import html
import laserembeddings
from fastcache import clru_cache
from laserembeddings import Laser
from typing import List, Union
from urllib.parse import unquote
from sklearn.model_selection import train_test_split

In [14]:
%%bash
# DOCS: https://github.com/facebookresearch/LASER/blob/master/install_models.sh

mkdir -p models/laser/
# for FILE in bilstm.eparl21.2018-11-19.pt eparl21.fcodes eparl21.fvocab bilstm.93langs.2018-12-26.pt 93langs.fcodes 93langs.fvocab; do
for FILE in bilstm.93langs.2018-12-26.pt 93langs.fcodes 93langs.fvocab; do
    wget -cq https://dl.fbaipublicfiles.com/laser/models/$FILE -O models/laser/$FILE
done

In [15]:
config = {
    "laser": {
        "base_dir":  "./models/laser",
        "bpe_codes": "./models/laser/93langs.fcodes",
        "bpe_vocab": "./models/laser/93langs.fvocab",
        "encoder":   "./models/laser/bilstm.93langs.2018-12-26.pt",
    }
}

# Instantiate encoder
# BUG: CUDA GPU memory is exceeded if both laser and labse are loaded together 
# @clru_cache(None)
def get_laser_model():
    laser_model = Laser(
        bpe_codes = config['laser']['bpe_codes'],
        bpe_vocab = config['laser']['bpe_vocab'],
        encoder   = config['laser']['encoder'],
        tokenizer_options = None,
        embedding_options = None
    )
    return laser_model
def laser_encode(text: Union[str, List[str]], lang='en', normalize=True) -> np.ndarray:
    """
    Encodes a corpus of text using LASER
    :param text: Large block of text (will be tokenized), or list of pre-tokenized sentences
    :param lang: 2 digit language code (optional autodetect)
    :return:     embedding matrix
    """
    laser_model = get_laser_model()
    
    # lang = lang or language_detect(text, threshold=0.0)
    if isinstance(text, str):
        # sentences = punkt_tokenize_sentences(text, lang=lang)
        sentences = [ text ]
    else:
        sentences = list(text)

    embedding = laser_model.embed_sentences(sentences, lang=lang)
    
    if normalize:
        embedding = embedding / np.sqrt(np.sum(embedding**2, axis=1)).reshape(-1,1)
        
    return embedding

In [45]:
# import datasets 
# dataset = datasets.load_dataset('ucberkeley-dlab/measuring-hate-speech', 'binary')   
# df = dataset['train'].to_pandas()
# df.describe()

ModuleNotFoundError: No module named 'datasets'

In [96]:
# network.summary()
df1 = pd.read_csv("dataset_multi.csv", delimiter=",", encoding='utf-8')

# print(col)
l=['hate_speech_score','text','target_race','target_religion','target_origin','target_gender','target_sexuality','target_age','target_disability']
df1=df1[l]
col=list(df1.columns)
print(df1)


        hate_speech_score                                               text  \
0                   -3.90  Yes indeed. She sort of reminds me of the elde...   
1                   -6.52  The trans women reading this tweet right now i...   
2                    0.36  Question: These 4 broads who criticize America...   
3                    0.26  It is about time for all illegals to go back t...   
4                    1.54  For starters bend over the one in pink and kic...   
...                   ...                                                ...   
135551              -4.88  عاجل سماحة #السيد_عبدالملك_بدرالدين_الحوثي  نص...   
135552              -4.40  Millions of #Yemen-is participated in mass ral...   
135553              -2.49  @AbeShinzo @realDonaldTrump @shinzoabe 独裁者は行きま...   
135554              -4.40  Millions of #Yemen-is participated in mass ral...   
135555              -0.20  لا تتشمت الرجال مسكين يعاني كس امه 😂. يقول يال...   

        target_race  target_religion  t

In [97]:
df1.loc[df1['hate_speech_score'] >= 0.5,'hate_speech_score' ] = 1
df1.loc[df1['hate_speech_score'] <0.5,'hate_speech_score'] = 0
# df1=df1[['text','hate_speech_score']]
# print(df1)
df1.rename(columns={'text':'comment','hate_speech_score':'isHate'},inplace=True)
df1=df1[df1['isHate']==1]
print(df1)


        isHate                                            comment  \
4          1.0  For starters bend over the one in pink and kic...   
7          1.0         Fuck off you insufferable retarded faggot.   
14         1.0  White Small Little Invisible Clits Are A Disgr...   
17         1.0  @suddenlywestan Because you are a country of m...   
18         1.0  This sub is full of grade A faggots  Comedy fu...   
...        ...                                                ...   
122428     1.0  STOP SUCKING THAT FATASS' DICK, YOU NIGGERLOVI...   
122429     1.0  Stupid Asian Bitch You're an idiot and you sho...   
122430     1.0  Stupid Asian Bitch You're an idiot and you sho...   
122431     1.0                         Fucking fagot burn in hell   
122434     1.0  Niggers and their liberal friends steal everyt...   

        target_race  target_religion  target_origin  target_gender  \
4             False            False          False           True   
7             False            

In [98]:
col=col[2:]
for i in col:
    df1.loc[df1[i] == 'True', i] = 1.0
    df1.loc[df1[i] == 'False', i] = 0.0
#     df1[i]=df1[i].apply(pd.to_numeric)
    df1[i] = df1[i].astype(int)
col.append("comment")
df1=df1[col]
df1=df1.reset_index(drop=True)

# print(df1.comment)
processed_tweets = preprocess(df1.comment)   
df1['processed_tweets'] = processed_tweets
df1 = df1.sample(frac = 1)
# print(df1)
word_tokenizer = Tokenizer()                      
word_tokenizer.fit_on_texts(df1["processed_tweets"])
# print(list(df1["processed_tweets"]))
brown_tagged_words=list(df1["processed_tweets"])
brown_tagged_words_encoded =word_tokenizer.texts_to_sequences(brown_tagged_words)
MAX_SEQ_LENGTH = max(lengths)
brown_tagged_words_padded = pad_sequences(brown_tagged_words_encoded, maxlen=MAX_SEQ_LENGTH, padding="post", truncating="post")
print(len(brown_tagged_words_padded[0]))
# brown_tagged_words_encoded = word_tokenizer.texts_to_sequences(df1["processed_tweets"])
col.remove('comment')
df2=df1[col]
# print(df2)
brown_tagged_vect=df2.to_numpy()
print(brown_tagged_vect.shape)

66
(49273, 7)


In [91]:
print(df1.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49273 entries, 42477 to 15417
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   target_race        49273 non-null  int32 
 1   target_religion    49273 non-null  int32 
 2   target_origin      49273 non-null  int32 
 3   target_gender      49273 non-null  int32 
 4   target_sexuality   49273 non-null  int32 
 5   target_age         49273 non-null  int32 
 6   target_disability  49273 non-null  int32 
 7   comment            49273 non-null  object
 8   processed_tweets   49273 non-null  object
dtypes: int32(7), object(2)
memory usage: 2.4+ MB
None


In [61]:
lengths = [len(seq) for seq in brown_tagged_words_encoded]
print("Length of longest sentence: {}".format(max(lengths)))
MAX_SEQ_LENGTH = max(lengths)
# brown_tagged_words_padded = pad_sequences(brown_tagged_words_encoded, maxlen=MAX_SEQ_LENGTH, padding="post", truncating="post")
# print(len(brown_tagged_words_padded[0]))
# print((word_tokenizer.word_index) )
path = "GoogleNews-vectors-negative300.bin"
word2vec= KeyedVectors.load_word2vec_format(path, binary=True)
EMBEDDING_SIZE  = 300
VOCABULARY_SIZE = len(word_tokenizer.word_index) + 1
embedding_weights = np.zeros((VOCABULARY_SIZE, EMBEDDING_SIZE))
word2id = word_tokenizer.word_index
for word, index in word2id.items():
    try:
        embedding_weights[index, :] = word2vec[word]
    except KeyError:
        pass

Length of longest sentence: 66


In [ ]:
len_brown_data=len(brown_tagged_words_padded)/5
avg_loss=0.0
avg_acc=0.0
confusion_mat=[]
per_pos_acc=[]
Y_pred=[]
Y_true=[]
# temp = list(zip(brown_tagged_words_padded,brown_tagged_vect ))
# random.shuffle(temp)
# brown_tagged_words_padded,brown_tagged_vect= zip(*temp)
# # res1 and res2 come out as tuples, and so must be converted to lists.
# brown_tagged_words_padded,brown_tagged_vect = list(brown_tagged_words_padded), list(brown_tagged_vect)
# print(brown_tagged_vect)
# print(len(brown_tagged_words_padded))
# for i in range(5):
# first_part=int(i*(len_brown_data))
# second_part=int((i+1)*len_brown_data)
# X_train=np.concatenate((brown_tagged_words_padded[:first_part],brown_tagged_words_padded[second_part:]),axis=0)
# X_test =brown_tagged_words_padded[first_part:second_part]
# Y_train=np.concatenate((brown_tagged_vect[:first_part],brown_tagged_vect[second_part:]),axis=0)
# Y_test =brown_tagged_vect[first_part:second_part]

#     Y_test_enc=brown_tagged_tags_encoded[first_part:second_part]
brown_tagged_words_padded = np.asarray(brown_tagged_words_padded).astype('float32')
brown_tagged_vect = np.asarray(brown_tagged_vect).astype('float32')

X_train, X_test, Y_train, Y_test = train_test_split(brown_tagged_words_padded, brown_tagged_vect, test_size=0.2, random_state=42)
X_train,X_validation=train_test_split(X_train,train_size=0.80,test_size=0.20,random_state = 50)
Y_train,Y_validation=train_test_split(Y_train,train_size=0.80,test_size=0.20,random_state = 50)
print(X_train.shape,Y_train.shape)
#     print(X_validation.shape)
network = models.Sequential()
#     Embedding(input_dim     = VOCABULARY_SIZE,
#                          output_dim    = EMBEDDING_SIZE,
#                          input_length  = MAX_SEQ_LENGTH,
# #                          weights       = [embedding_weights],
#                          ),
#     Bidirectional(LSTM(16)),
    
# #     tf.keras.Input(shape=(,)),
#     tf.keras.layers.Dense(512, activation=tf.keras.layers.LeakyReLU(alpha=0.1)),
#         tf.keras.layers.BatchNormalization(),
#         tf.keras.layers.Dropout(0.25),
#     tf.keras.layers.Dense(128, activation=tf.keras.layers.LeakyReLU(alpha=0.1)),
#         tf.keras.layers.BatchNormalization(),
#         tf.keras.layers.Dropout(0.25),
#     tf.keras.layers.Dense(32, activation=tf.keras.layers.LeakyReLU(alpha=0.1)),
#         tf.keras.layers.BatchNormalization(),
#         tf.keras.layers.Dropout(0.25),
# #     tf.keras.layers.Dense(8, activation=tf.keras.layers.LeakyReLU(alpha=0.1)),
# #         tf.keras.layers.BatchNormalization(),
# #         tf.keras.layers.Dropout(0.25),
#     tf.keras.layers.Dense(8, activation=tf.keras.activations.sigmoid),
# ]
#     ) ## starting the ffnn
network.add(Embedding(input_dim     = VOCABULARY_SIZE,
                         output_dim    = EMBEDDING_SIZE,
                         input_length  = MAX_SEQ_LENGTH,
#                          weights       = [embedding_weights],
                         ))
network.add(Bidirectional(LSTM(16))) 
#     network.add(layers.Dense(units=100))
#     network.add(layers.Dense(units=1, activation='sigmoid'))
#     network.add(layers.Dense(units=1,activation='relu'))
network.add(layers.Dense(512, kernel_regularizer='l1'))
network.add(layers.BatchNormalization())
network.add(layers.Dropout(0.3))
network.add(layers.Dense(7, activation='sigmoid'))

network.compile(loss='categorical_crossentropy', # Cross-entropy
            optimizer=tf.keras.optimizers.Adam(learning_rate=5e-4), # Root Mean Square Propagation
            metrics=['categorical_accuracy'])
trainng = network.fit(X_train,Y_train,batch_size=512,epochs=10,validation_data=(X_validation, Y_validation))
loss,accuracy = network.evaluate(X_test,Y_test,verbose=1)
print(loss)
print(accuracy)
avg_loss=avg_loss+loss
avg_acc=avg_acc+accuracy
pred=network.predict(X_test)
print(pred)
print(Y_test)
# print(avg_acc/5)
#     print(avg_acc)
#     indices=np.argmax(pred, axis=2)
    #not_zero=indices[indices != 0]
#     Y_pred=indices 

(31534, 66) (31534, 7)
Epoch 1/10
62/62 [==============================] - 107s 2s/step - loss: 8.3838 - categorical_accuracy: 0.5887 - val_loss: 6.7529 - val_categorical_accuracy: 0.5828
Epoch 2/10
62/62 [==============================] - 93s 2s/step - loss: 6.1832 - categorical_accuracy: 0.6834 - val_loss: 4.5129 - val_categorical_accuracy: 0.6523
Epoch 3/10
62/62 [==============================] - 88s 1s/step - loss: 5.5478 - categorical_accuracy: 0.6934 - val_loss: 3.9151 - val_categorical_accuracy: 0.6877
Epoch 4/10
31/62 [==============>...............] - ETA: 42s - loss: 5.5223 - categorical_accuracy: 0.7068

In [ ]:
# print(word_tokenizer.word_index['UNK'])
sentence=input()
# print(word_tokenizer.word_index)

def sen_vec(sent):
    sent=preprocess(sent)
    sent=str(sent[0])
    print(sent)
    y=len(word_tokenizer.word_index)
    words=sent.split(" ")
    print(words)
    l = []
    for i in range(len(words)):
        if words[i] in word_tokenizer.word_index:
            l.append(word_tokenizer.word_index[words[i]])
        else:
            print(words[i])
            try:
                sim_word = word2vec.most_similar(positive=words[i],topn=10)[:]
                print(sim_word)
                x=0
                for i in range(10):
                    x=x+1
                    if sim_word[i][0] in word_tokenizer.word_index:
                        l.append(word_tokenizer.word_index[sim_word[i][0]])
                        break
                if(x==10):
#                     print("y is not in the vocalbulary")
#                     print("y is not in the vocalbulary")
                        l.append(0)
            
            except KeyError:
#                 sys.exit("y is not in the vocalbulary")
                
#                 y=y+1
                l.append(0)
                
            
#         x=0
#         for j in range(len(brown_tagged_words_encoded)):
#             for k in range(len(brown_tagged_words_encoded[j])):
#                 if(x==0 and words[i]==brown_tagged_words[j][k]):
#                     l.append(int(brown_tagged_words_encoded[j][k]))
#                     x=1
#                     break
#             if(x==1):
#                 break
#         if(x==0):
#             sim_word = word2vec.most_similar(positive=words[i],topn=10)[:][0]
#             while(x==0):
#                 for j in range(len(brown_tagged_words_encoded)):
#                     for k in range(len(brown_tagged_words_encoded[j])):
#                         if(x==0 and words[i]==brown_tagged_words[j][k]):
#                             l.append(int(brown_tagged_words_encoded[j][k]))
#                             x=1
#                             break
#                     if(x==1):
#                         break
        
              
                            
                        
                
    t=tf.convert_to_tensor(l)
    t=tf.reshape(t, [1, t.shape[0]])
    y = pad_sequences(t, maxlen=MAX_SEQ_LENGTH, padding="pre", truncating="post")
    return y

In [ ]:
l=[]
# sentence=sentence
df = pd.DataFrame(columns=['sentence'])
value1 = "dog"
df = df.append({'sentence': sentence}, ignore_index=True)
# from io import StringIO
# StringData = StringIO(sentence)
 
# # let's read the data using the Pandas
# # read_csv() function
# df = pd.read_csv(StringData, sep =";")
 
# Print the dataframe
# print(df)
# print(df.sentence)
l=sen_vec(df.sentence)
print("yes")
pred=network.predict(l)
print(pred)

In [ ]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2),max_df=0.75, min_df=5, max_features=10000)

# TF-IDF feature matrix
tfidf = tfidf_vectorizer.fit_transform(df['processed_tweets'] )

In [ ]:
print(5)

In [ ]:
X = tfidf
# tfidf_a = tfidf.toarray()
print(tfidf_a)
y = df['isHate'].astype(int)
# X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)
# X_train_tfidf

In [ ]:
lr = LogisticRegression()
rf = RandomForestClassifier()
xgb = XGBClassifier()
# %%time
accuracy = []
skf = StratifiedKFold(n_splits=10)
skf.get_n_splits(X, y)
for clf in (lr,rf,xgb):

    StratifiedKFold(n_splits=10, random_state=None, shuffle=False)
    for train_index, test_index in skf.split(X, y):
            
            X1_train, X1_test = X[train_index], X[test_index]
            y1_train, y1_test = y[train_index], y[test_index]

            model = clf.fit(X1_train,y1_train)
            y_preds = model.predict(X1_test)
            acc = (sklearn.metrics.f1_score(y1_test,y_preds,average = 'micro'))
            accuracy.append(acc)
    print(clf.__class__.__name__,np.array(accuracy).mean())

In [ ]:
rf=LogisticRegression()
X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)
rf.fit(X_train_tfidf,y_train)
y_preds = rf.predict(X_test_tfidf)
acc1=accuracy_score(y_test,y_preds)
report = classification_report( y_test, y_preds )
print(y_test)
print(y_preds)
print(report)
print("Random Forest, Accuracy Score:",acc1)